In [23]:
#import libraries 
import sys
import matplotlib.pyplot as plt # plotting
plt.style.use('ggplot')
import numpy as np #linear alg
import os # accessing directory structure
import pandas as pd #CSV data 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
import nltk
nltk.download('vader_lexicon')

import warnings
warnings. filterwarnings("ignore")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/arthurt/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [24]:
df1 = pd.read_csv('AppleFinalData.csv')

In [25]:
df2 = pd.read_csv('AppleNewsStock.csv')

In [26]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
sid = SentimentIntensityAnalyzer()

neg, neu, pos, compound = [], [], [], []

def sentiment(df):
    for i in range(len(df)):
        sen = unicodedata.normalize('NFKD', train.iloc[i]['News'])
        ss = sid.polarity_scores(sen)
        neg.append(ss['neg'])
        neu.append(ss['neu'])
        pos.append(ss['pos'])
        compound.append(ss['compound'])
    df['neg'] = neg
    df['neu'] = neu
    df['pos'] = pos
    df['compound'] = compound
    
    """In summary, this code calculates sentiment scores 
    (negative, neutral, positive, and compound) for each piece of text in the 
    "News" column of the DataFrame df using the VADER sentiment analyzer and 
    adds these scores as new columns to the DataFrame."""

In [27]:
def if_news(column):
    if column ==0:
        return 0
    else:
        return 1

In [28]:
df1['if_news'] = df1['compound'].apply(if_news)

df_weekly = df1[['Date','Close', 'compound', 'neg', 'pos', 'if_news']]
df_weekly['Date'] = pd.to_datetime(df_weekly['Date'])
df_weekly.set_index('Date',inplace=True)
#creates a new column in df1 called 'if_news'it uses apply to apply if_news into each compound 
#column and they can be put into binary values where 0 indicates

In [29]:
def take_last(array_like):
    return array_like[-1]

In [30]:
output = df_weekly.resample('W', loffset=pd.Timedelta(days=-6)).agg({
    'Close': 'last',
    'compound': 'mean',
    'neg': 'mean',
    'pos': 'mean',
    'if_news': 'max'
})
# calculates mean for the weekly sentiment score for neg and pos, 
#then close means where the value 
# of the stock closed. 

In [31]:
output.head(10)

,Close,compound,neg,pos,if_news
Date,,,,,
2006-11-27,91.32,0.770700,0.032000,0.063000,1
2006-12-04,88.26,0.141400,0.043000,0.057400,1
2006-12-11,87.72,0.100180,0.040000,0.052400,1
2006-12-18,82.20,-0.007940,0.060800,0.061000,1
2006-12-25,84.84,0.483825,0.067500,0.096000,1
2007-01-01,85.05,0.475767,0.049333,0.078333,1
2007-01-08,94.62,0.141240,0.066000,0.071000,1
2007-01-15,88.50,0.468150,0.057000,0.105750,1
2007-01-22,85.38,0.597520,0.026000,0.078800,1


In [32]:
    """
    Perform k-fold cross-validation on the data
    
    Parameters
    ----------
    df: DataFrame containing the data with date as the index
    X: DataFrame containing the independent variable(s) (i.e., features, predictors)
    y: DataFrame containing the dependent variable (i.e., the target)
    k: Number of folds for cross-validation (default is 5)
    
    Optional
    --------
    seed: Integer used as the seed for a random number generator
          You don't necessarily NEED to use a random number generator, but if you do, please use the default value for seed
    
    Returns
    -------
    List of tuples: Each tuple contains (X_train, X_test, y_train, y_test) for one fold
    """

"\nPerform k-fold cross-validation on the data\n\nParameters\n----------\ndf: DataFrame containing the data with date as the index\nX: DataFrame containing the independent variable(s) (i.e., features, predictors)\ny: DataFrame containing the dependent variable (i.e., the target)\nk: Number of folds for cross-validation (default is 5)\n\nOptional\n--------\nseed: Integer used as the seed for a random number generator\n      You don't necessarily NEED to use a random number generator, but if you do, please use the default value for seed\n\nReturns\n-------\nList of tuples: Each tuple contains (X_train, X_test, y_train, y_test) for one fold\n"

In [33]:
def k_fold_split(df, X, y, k=5, seed=42):

    # Create the random number generator
    rng = np.random.RandomState(seed)
    
    # Create k-fold cross-validator
    kfold = KFold(n_splits=k, shuffle=True, random_state=rng)
    
    # Initialize list to store the train-test splits
    splits = []
    
    for train_idx, test_idx in kfold.split(df):
        # Split the data into train and test sets based on the index
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Add the current fold split to the list
        splits.append((X_train, X_test, y_train, y_test))
    
    return splits